In [1]:
import os
import warnings
import string
from time import time

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gensim
from gensim.models.doc2vec import Doc2Vec
from sentence_transformers import SentenceTransformer

from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english') + [word.strip("\n") for word in open("remove_words.txt", "r")])

In [2]:
print("---------------------------- LOADING MODEL---------------------------- ")
start = time()
global model
filename = 'doc2vec/doc2vec.bin'
model= Doc2Vec.load(filename)
print('MODEL LOADED in %.2f seconds' % (time() - start))

---------------------------- LOADING MODEL---------------------------- 
MODEL LOADED in 6.97 seconds


In [8]:
pd.set_option('display.max_rows', 1000)
relevant_df = pd.read_csv("relevant_test.csv", usecols = ['word','frequency','frequency_scaled','relevancy_score','weighted_score'])
relevant_df

,word,frequency,frequency_scaled,relevancy_score,weighted_score
0,tax,503,0.529474,41.757174,22.109325
1,retirement,481,0.506316,42.874622,21.708098
2,income,399,0.420000,48.428511,20.339975
3,pension,392,0.412632,45.992408,18.977920
4,plan,339,0.356842,50.089484,17.874037
5,rate,313,0.329474,52.577644,17.322950
6,financial,334,0.351579,44.533258,15.656956
7,saving,285,0.300000,50.692381,15.207714
8,pay,249,0.262105,51.742485,13.561978
9,money,274,0.288421,45.271841,13.057352


In [9]:
num_similar_words = 1
similarity_threshold = 0.7
words_to_add = []

for index, row in relevant_df.iterrows():
    print(row['word'])
    
    similar_words = model.most_similar(positive=[row['word']], topn = num_similar_words)
    
    print(similar_words)
    
    for similar_word in similar_words:
        if similar_word[0] != (row['word'] + 's') and similar_word[1] >= similarity_threshold:
            if (similar_word[0] not in words_to_add) and (similar_word[0] not in list(relevant_df['word'])):
                words_to_add.append(similar_word[0])
                temp_dict = {"word": similar_word[0], "frequency": row['frequency'], "frequency_scaled": row['frequency_scaled'], "relevancy_score": row['relevancy_score'], "weighted_score": row["weighted_score"]}
                relevant_df = relevant_df.append(temp_dict, ignore_index = True)
    print("----------------------------------------------------------------------")
    
relevant_df = relevant_df.sort_values(by=['weighted_score'], ascending=False)

tax
[('taxes', 0.9098928570747375)]
----------------------------------------------------------------------
retirement
[('retiring', 0.8236984014511108)]
----------------------------------------------------------------------
income
[('incomes', 0.7755939960479736)]
----------------------------------------------------------------------
pension
[('pensions', 0.77666836977005)]
----------------------------------------------------------------------
plan
[('plans', 0.845669686794281)]
----------------------------------------------------------------------
rate
[('rates', 0.8448569774627686)]
----------------------------------------------------------------------
financial
[('finance', 0.802301287651062)]
----------------------------------------------------------------------
saving
[('saved', 0.7769123911857605)]
----------------------------------------------------------------------
pay
[('paid', 0.8858240842819214)]
----------------------------------------------------------------------
money
[

In [10]:
relevant_df

,word,frequency,frequency_scaled,relevancy_score,weighted_score
0,tax,503,0.529474,41.757174,22.109325
70,taxes,503,0.529474,41.757174,22.109325
1,retirement,481,0.506316,42.874622,21.708098
71,retiring,481,0.506316,42.874622,21.708098
2,income,399,0.420000,48.428511,20.339975
3,pension,392,0.412632,45.992408,18.977920
4,plan,339,0.356842,50.089484,17.874037
5,rate,313,0.329474,52.577644,17.322950
6,financial,334,0.351579,44.533258,15.656956
72,finance,334,0.351579,44.533258,15.656956
